# IoT遙控車

In [10]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [17]:
%serialconnect to --port=com13 --baud=115200

Connecting to --port=com13 --baud=115200 
Ready.


### UDP測試

[設定方式](udp-control-setting.ipynb)

- 控制Led

- [Android控制器程式](https://play.google.com/store/apps/details?id=udpcontroller.nomal)

![remote-control-app1](image/remote-control-app1.png)

- [iphone版控制器程式](https://itunes.apple.com/us/app/udp-commander/id977326536?mt=8)

![remote-control-app2](image/remote-control-app2.png)

In [8]:
%sendtofile main.py

import socket
from machine import Pin
import time

p2 = Pin(2, Pin.OUT) #D4
p2.value(1)
for i in range(6):
    p2.value(not p2.value())
    time.sleep(0.1)
    
address = ('0.0.0.0', 1234)
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.bind(address)

print('waiting...')
while True:
    data, addr = s.recvfrom(1)
    print('received:',data,'from',addr)
    #s.sendto(data, addr) #echo
    if(data[:1]==b'f'):
        p2.value(not p2.value())
        time.sleep(0.1)        

Sent 22 lines (482 bytes) to main.py.


# 自製遙控車

In [9]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time
import socket

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())
    time.sleep(0.2)
    
# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1
REV = 0

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(0)     #r
    #motor_a2.value(FWD)
    motor_b1.value(1)     #l
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(0)
    #motor_b2.value(REV)

# 快速右轉
def car_right2():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)    

# 快速左轉
def car_left2():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58
    
# go~
address = ('0.0.0.0', 1234)
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.bind(address)

#
car_left()
time.sleep(0.5)
car_right()
time.sleep(0.5)
car_stop()

MOVE_DELAY = 0.1
while(True):
    car_stop()
    #distance=ping(trigPin=D7,echoPin=D8)
    #print("d=%d" %(distance))
    
    data, addr = s.recvfrom(1234)
    print('received:',data,'from',addr)
    if(data[:1]==b'f'):
        car_fwd()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'b'):
        car_rev()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'r'):
        car_right()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'l'):
        car_left()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'R'):
        car_right2()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'L'):
        car_left2()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b's'):
        car_stop()
        time.sleep(MOVE_DELAY)


Sent 153 lines (3284 bytes) to main.py.


----

### 設計你的特技動作

- 搖頭
- 轉小圈、轉大圈
- 一鍵跑8字
- ...etc

In [ ]:
# TODO:

----

## 資訊探勘遙控車 

回傳前方距離為範例

In [14]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [15]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time
import socket

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())
    time.sleep(0.2)
    
# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1
REV = 0

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(0)     #r
    #motor_a2.value(FWD)
    motor_b1.value(1)     #l
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(0)
    #motor_b2.value(REV)

# 快速右轉
def car_right2():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)    

# 快速左轉
def car_left2():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58
    
# go~
address = ('0.0.0.0', 1234)
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.bind(address)

#
car_left()
time.sleep(0.5)
car_right()
time.sleep(0.5)
car_stop()

MOVE_DELAY = 0.1
while(True):
    car_stop()
    #distance=ping(trigPin=D7,echoPin=D8)
    #print("d=%d" %(distance))
    
    data, addr = s.recvfrom(1234)
    print('received:',data,'from',addr)
    if(data[:1]==b'f'):
        car_fwd()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'b'):
        car_rev()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'r'):
        car_right()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'l'):
        car_left()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'R'):
        car_right2()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b'L'):
        car_left2()
        time.sleep(MOVE_DELAY)
    if(data[:1]==b's'):
        car_stop()
        time.sleep(MOVE_DELAY)
    
    distance=ping(trigPin=D7,echoPin=D8)
    s.sendto(str(distance).encode(), (addr[0], 1234))


Sent 156 lines (3383 bytes) to main.py.
